In [36]:
# Make sure to install these libraries
# pip install numpy
# pip install glob2
# pip install opencv-python

import numpy as np
import skimage.transform
import glob
import cv2
import math
import os

In [37]:
# Variables
crop = 1
adjust = 1

# Instead of hard-coding, implement a file finder
# Change the directory specific to your computer
# If you're using Windows, use 2 backslashes (\) instead of 1 foward slash (/)
direc = '/home/jeff/Documents/Ultrascan/Ultrascan_Py/Plantar/Scan_3d__bmp/'

padSide = 0
numSlices = 512  # Number of rotational slices
rotTot = 255     # Total rotation sweep in degrees
probeID = 0
imgW = 640       # slice image width
imgH = 480       # slice image height
thic = 3         # Thicknes of slice in pixels
interval = 1     # Interval between slices, 1 => use all slices
imgDepth = 12    # Depth of scan in cm
dia = 16         # Diameter of circle in cm
filtSize1 = 1
filtSize2 = 1

In [38]:
# Opens the file and store the text in a 2d array
direc_to_data_txt = direc + 'Scan_3d__data.txt'

fileID = open(direc_to_data_txt)
data_txt_2d = np.genfromtxt(fileID,dtype='str')
fileID.close()

for i in range(len(data_txt_2d)):
    for j in range(len(data_txt_2d[i])):
        print(data_txt_2d[i][j], end=" ")
    print()

Index 1 
Depth 120 
Frequency: 7500 
High_Voltage: 50 
Focus: 60 
Main_Gain: -1.25 
Dynamic: 65 
TGC_Near: 0 
TGC_Middle: 0 
TGC_Far: -40 
Doubler: True 
Compound: False 
Compound_Angle: 0 
Steering: 0 
Number_of_Lines: 127 
Probe_ID: 512 
Width: 640 
Height: 480 
Number_Images: 588 
Sweep_Angle: 256 
Radius: 7.75 


In [39]:
probeID = int(data_txt_2d[15][1])
imgW = int(data_txt_2d[16][1])
imgH =  int(data_txt_2d[17][1])
imgDepth = int(data_txt_2d[1][1]) / 10
rotTot = int(data_txt_2d[19][1])
dia = float(data_txt_2d[20][1]) * 2

# print(probeID)
# print(imgW)
# print(imgH)
# print(depth)
# print(rotTot)
# print(dia)

In [40]:
# This is to crop the sides of the image in the conical shape for 7.5 Hz
# b is the value that corresponds to guardband and cone edges
# m is the value that corresponds to the diagonal
b = 70 # these b and m values are the original ones that Austeen had
m = .5
gband = 0

if (probeID == 512):
    if (imgDepth == 4):
        b = 0;
        m = 0
    elif (imgDepth == 8):
        b = 70
        m = 0.5
    elif (imgDepth == 12):  
        b = 0
        m = 0
    elif (imgDepth == 16): 
        b = 0
        m = 0
elif (probeID == 256): 
    if (imgDepth == 2):
        b = 320
    elif (imgDepth == 4):
        b = 200 + gband  
    elif (imgDepth == 6):   
        b = 132.5 + gband; 
    elif (imgDepth == 8):
        b = 100 + gband
    elif (imgDepth == 10): 
        b = 80 + gband

In [41]:
# Taking the first part of the image to crop the top area 
# where it is convex
cropTop = round(imgH / (imgDepth * 4))

# Bottom tangent line to the circle of imaging
# will be used to crop the bottom of the image
cropBottom = imgH - round((imgDepth - dia / 2) * imgH / imgDepth)

# Pixels to pad array for rotation
# so that we do not lose any data while slicing
cropPadSide = round((dia - imgDepth) * imgH / imgDepth)

if (cropPadSide < 0):
    # Only in case of 16cm imaging - need to pad sides to prevent loss
    cropPadSide = abs(cropPadSide)
    padSide = 1
else:
    padSide = 0

In [42]:
# Pixels to take off sides
# Based on the C# code in the software sdk we have settings of 4cm interval
# We can adjust the C# code and get intervals of 2cm as well
imgW = 640 
cropSide_d5 = 0
startPix = 0

'''
if (imgDepth == 4):
    imgW = 640
    startPix = 80 # What is startPix? I don't see it initialized anywhere else
if (imgDepth == 8):
    imgW = 640
    startPix = 200
if (imgDepth == 12):
    imgW = 640
    startPix = 240
if (imgDepth == 16):
    imgW = 640
    startPix = 260
'''

"\nif (imgDepth == 4):\n    imgW = 640\n    startPix = 80 # What is startPix? I don't see it initialized anywhere else\nif (imgDepth == 8):\n    imgW = 640\n    startPix = 200\nif (imgDepth == 12):\n    imgW = 640\n    startPix = 240\nif (imgDepth == 16):\n    imgW = 640\n    startPix = 260\n"

In [43]:
# For our knowledge
if (crop == 1):
    print("Cropping Images: ON")
else:
    print("Cropping Images: OFF")

# For our knowledge
if (adjust == 1):
    print("Adjusting Images: ON")
else:
    print("Adjusting Images: OFF")
    
print("Always adjust the number of silces analyzed and always adjust the rotation angle of the images")

Cropping Images: ON
Adjusting Images: ON
Always adjust the number of silces analyzed and always adjust the rotation angle of the images


In [44]:
# Places all the file paths to the images in nf
direc_to_bitmaps = direc + 'Scan_3d__bmp/' + 'Image_*_*_*.bmp'
nf = glob.glob(direc_to_bitmaps)

# Sorts nf by image number
image_number = [0] * len(nf)

for i in range(len(nf)):
    image_number[i] = int(nf[i].replace(direc + "Scan_3d__bmp/Image_", "").split("_")[0])
    
image_number = np.array(image_number)
indices = np.argsort(image_number)
nf_placeholder = nf.copy() # Without copy, it would just point back to nf

for i in range(len(indices)):
    nf[i] = nf_placeholder[indices[i]]
    
numSlices = len(nf)

print("Directory: " + direc)
print("Number of slices: " + str(numSlices))
print("Diameter: " + str(dia))
print("Rotation Angle: " + str(rotTot))

rotInc = rotTot / numSlices # Degrees rotated between slices

Directory: /home/jeff/Documents/Ultrascan/Ultrascan_Py/Plantar/Scan_3d__bmp/
Number of slices: 588
Diameter: 15.5
Rotation Angle: 256


In [45]:
# Creates a 3d array of zeroes
summed = np.uint8(np.zeros((imgH + cropPadSide, imgW, imgH + cropPadSide)))
[L,W,H]= summed.shape

print("Reconstructing...")

Reconstructing...


In [46]:
# Testing to see if the grayscaled image is the same as the
# non-grayscaeled image

'''
filename = nf[0]
rawImg1 = np.array(cv2.imread(filename, cv2.IMREAD_GRAYSCALE))
rawImg2 = np.array(cv2.imread(filename))

rawImg = np.array(rawImg1)
row, col = rawImg.shape

for row in range(0, int(row)):
    for col in range(0, int(col)):
        if (rawImg1[row][col] != (rawImg2[row][col])[0]):
            print("Failed")
            break
'''

'\nfilename = nf[0]\nrawImg1 = np.array(cv2.imread(filename, cv2.IMREAD_GRAYSCALE))\nrawImg2 = np.array(cv2.imread(filename))\n\nrawImg = np.array(rawImg1)\nrow, col = rawImg.shape\n\nfor row in range(0, int(row)):\n    for col in range(0, int(col)):\n        if (rawImg1[row][col] != (rawImg2[row][col])[0]):\n            print("Failed")\n            break\n'

In [47]:
# vvv
# vvv
# vvv
# REMINDER DO SUBTRACT 1 FOR THE ARRAYS SINCE MATLAB INDEXES FROM 1
# MATLAB LOOPS TO END INCLUSIVE
# ^^^
# ^^^
# ^^^

startPix = 0

for i in range(0, numSlices, interval):
    filename = nf[i]
    rawImg = np.array(cv2.imread(filename, cv2.IMREAD_GRAYSCALE))
    
    # Size of the image before cropping - always 480x640
    SImg1 = rawImg.shape
    height, width = rawImg.shape
    
    # if (adjust == 1):
        # rawImg = imguidedfilter()
        # rawImg = imadjust()
    
    if (crop == 1):
        if (probeID == 512):
            rawImg[0: cropTop - 1, : ] = 0
            rawImg[cropBottom - 1: height - 1, : ] = 0
            # Make a much smaller raw image after cropping
            # the top and the bottom components
            rawImg = rawImg[ : , startPix : startPix + imgW]
    
    # Crop the image into a triangle/cone shape while cutting off the sides
    # Use top width and bottom width and shave off the sides
    # of the image turning it into a polygon
    if (probeID == 512):
        for q in range(0, imgH):
            cropTriangle = round(imgW / 2 - (m * q + b))
            rawImg[q, 0 : cropTriangle - 1] = 0
            rawImg[q, imgW - cropTriangle - 1: imgW - 1] = 0
    elif (probeID == 256):
        for q in range(0, imgH):
            cropSide = round(imgW / 2 - b)
            rawImg[q, 0 : cropSide - 1] = 0
            rawImg[q, imgW - cropSide - 1: imgW - 1] = 0
    
    # Filtering using an averaging method to make brightness - 1
    # for bad areas
    rawImg[0, : ] = -1
    
    # New width and height in case rawImg got cropped
    height, width = rawImg.shape
    
    # Filtering using an averaging method to make brightness -1 for all bad areas
    for k in range(0, width):
        for m in range(0, height - 6):
            if ((rawImg[m + 6 - 1, k - 1] / 7 + rawImg[m + 5 - 1, k - 1] / 7 +
               rawImg[m + 4 - 1, k - 1] / 7 + rawImg[m + 3 - 1, k - 1] / 7 +
               rawImg[m + 2 - 1, k - 1] / 7 + rawImg[m + 1 - 1, k - 1] / 7) > 90):
                break
            rawImg[m - 1, k - 1] = -1
            
    # Size of raw image after the cropping
    SImg2 = rawImg.shape
    
    # Only PadSide for 16cm arrays and greater
    # Initialize 3d array to contain slice
    temp = np.uint8(np.zeros((imgH + cropPadSide, imgW, (2 * thic) + 1)))
    
    for j in range(-thic - 1, thic):
        if (padSide == 0):
            temp[ : , : , thic + 1 + j - 1] = np.vstack((rawImg, np.zeros((cropPadSide, imgW))))
        else:
            temp[ : , : , thic + 1 + j - 1] = np.vstack((np.zeros((cropPadSide, imgW)), rawImg))
            
    # Permute 3d slice, rotate about vertical axis, permute back
    theta = rotInc * (i - 1)
    temp = np.transpose(skimage.transform.rotate(np.transpose(temp, (0, 2, 1)), theta), (0, 2, 1))
    
    L2, W2, H2 = temp.shape
    
    if (L2 > L):
        temp = temp[0: L - 1, : , : ]
        L2 = L
    if (H2 > H):
        temp = temp[ : , : , 0: H - 1]
        
    off1 = math.floor((L - L2) / 2)
    off2 = math.floor((H - H2) / 2)
    
    # Getting the max value from summed[0 + off1: L2 - 1 + off1, : , 0 + off2: H2 - 1 + off2]
    # and placing that value for all summed[0 + off1: L2 - 1 + off1, : , 0 + off2: H2 - 1 + off2]
    max_value = np.amax(summed[0 + off1: L2 - 1 + off1, : , 0 + off2: H2 - 1 + off2])
    summed[0 + off1: L2 - 1 + off1, : , 0 + off2: H2 - 1 + off2] = max_value
    
    # Output slice number
    print("Slice number: " + str(i + 1))

Slice number: 1
Slice number: 2
Slice number: 3
Slice number: 4
Slice number: 5
Slice number: 6
Slice number: 7
Slice number: 8
Slice number: 9
Slice number: 10
Slice number: 11
Slice number: 12
Slice number: 13
Slice number: 14
Slice number: 15
Slice number: 16
Slice number: 17
Slice number: 18
Slice number: 19
Slice number: 20
Slice number: 21
Slice number: 22
Slice number: 23
Slice number: 24
Slice number: 25
Slice number: 26
Slice number: 27
Slice number: 28
Slice number: 29
Slice number: 30
Slice number: 31
Slice number: 32
Slice number: 33
Slice number: 34
Slice number: 35
Slice number: 36
Slice number: 37
Slice number: 38
Slice number: 39
Slice number: 40
Slice number: 41
Slice number: 42
Slice number: 43
Slice number: 44
Slice number: 45
Slice number: 46
Slice number: 47
Slice number: 48
Slice number: 49
Slice number: 50
Slice number: 51
Slice number: 52
Slice number: 53
Slice number: 54
Slice number: 55
Slice number: 56
Slice number: 57
Slice number: 58
Slice number: 59
Slice 

Slice number: 463
Slice number: 464
Slice number: 465
Slice number: 466
Slice number: 467
Slice number: 468
Slice number: 469
Slice number: 470
Slice number: 471
Slice number: 472
Slice number: 473
Slice number: 474
Slice number: 475
Slice number: 476
Slice number: 477
Slice number: 478
Slice number: 479
Slice number: 480
Slice number: 481
Slice number: 482
Slice number: 483
Slice number: 484
Slice number: 485
Slice number: 486
Slice number: 487
Slice number: 488
Slice number: 489
Slice number: 490
Slice number: 491
Slice number: 492
Slice number: 493
Slice number: 494
Slice number: 495
Slice number: 496
Slice number: 497
Slice number: 498
Slice number: 499
Slice number: 500
Slice number: 501
Slice number: 502
Slice number: 503
Slice number: 504
Slice number: 505
Slice number: 506
Slice number: 507
Slice number: 508
Slice number: 509
Slice number: 510
Slice number: 511
Slice number: 512
Slice number: 513
Slice number: 514
Slice number: 515
Slice number: 516
Slice number: 517
Slice numb

In [49]:
# Export the newly created slices
nanInd = np.isnan(summed)
# summed2 = summed.astype('float32') # 32 bit vs 64 bit - less memory used
# summed2[nanInd] = -1000

convert = np.amax(np.amax(np.amax(summed))) / 255

# Creating X,Y, and Z folders and putting the sliced images into those
print("Outputting New Slices...")

Outputting New Slices...


In [ ]:
# Creating SlicesZ
# This block of code transposes summed before getting the for loops
# Does a 3d tranpose

try:
    os.mkdir(direc + "NewSlicesZ")
except FileExistsError as e:
    print("NewSlicesZ was already created.")
    
summed1 = np.transpose(summed, (0, 2, 1))
L, W, H = summed1.shape

for j in range(0, W):
    filename2 = direc + "NewSlicesZ/" + "Image_" + '{0:03}'.format(j) + ".bmp"
    cv2.imwrite(filename2, summed1[ : , j, : ])

# Creating SlicesX
try:
    os.mkdir(direc + "NewSlicesX")
except FileExistsError as e:
    print("NewSlicesX was already created.")

summed2 = np.transpose(summed, (1, 0, 2))
L, W, H = summed2.shape

for j in range(0, H):
    filename2 = direc + "NewSlicesX/" + "Image_" + '{0:03}'.format(j) + ".bmp"
    cv2.imwrite(filename2, summed2[ : , : , j])

# Creating SlicesY
try:
    os.mkdir(direc + "NewSlicesY")
except FileExistsError as e:
    print("NewSlicesY was already created.")

summed3 = np.transpose(summed, (1, 2, 0))
L, W, H = summed3.shape

for j in range(0, L):
    filename2 = direc + "NewSlicesY/" + "Image_" + '{0:03}'.format(j) + ".bmp"
    cv2.imwrite(filename2, summed3[j , : , :])

In [50]:
# Creating SlicesZ
# This block of code transposes summed in the for loop
# Does a 2d transpose
try:
    os.mkdir(direc + "NewSlicesZ")
except FileExistsError as e:
    print("NewSlicesZ was already created.")
    
for j in range(0, W):
    filename2 = direc + "NewSlicesZ/" + "Image_" + '{0:03}'.format(j) + ".bmp"
    cv2.imwrite(filename2, np.transpose(summed[ : , j, : ], (1, 0)))
    
# Creating SlicesX
try:
    os.mkdir(direc + "NewSlicesX")
except FileExistsError as e:
    print("NewSlicesX was already created.")
    
for j in range(0, H):
    filename2 = direc + "NewSlicesX/" + "Image_" + '{0:03}'.format(j) + ".bmp"
    cv2.imwrite(filename2, np.transpose(summed[ : , : , j], (1, 0)))

# Creating SlicesY
try:
    os.mkdir(direc + "NewSlicesY")
except FileExistsError as e:
    print("NewSlicesY was already created.")
    
for j in range(0, L):
    filename2 = direc + "NewSlicesY/" + "Image_" + '{0:03}'.format(j) + ".bmp"
    cv2.imwrite(filename2, np.transpose(summed[j , : , :], (1, 0)))

NewSlicesZ was already created.
NewSlicesX was already created.
NewSlicesY was already created.
620
(620, 640, 620)
